In [37]:
#calculates probabilities for tickers
import requests
from lxml import html
from datetime import datetime, date
import numpy as np
from collections import namedtuple
from ipynb.fs.full.option_classes import *

ticker ='tdc'
url = f'https://finance.yahoo.com/quote/{ticker.lower()}/options'
response = requests.get(url)
tree = html.fromstring(response.content)


In [2]:
strikes = tree.xpath('//td[@class="data-col2 Ta(end) Pstart(7px)"]/a')
[i.text for i in strikes]
contractName = [i.text for i in tree.xpath("//td[contains(@class,'data-col0 Ta(start) Pstart')]/a")]
strike, changePrice, changePercent= \
  ([i.text for i in tree.xpath(f'//td[contains(@class,"data-col{i} Ta(end) Pstart")]/a')] for i in [ 2, 6, 7])
lastTradeDate, lastPrice, bid, ask, volume, openInterest, impliedVol = \
  ([i.text for i in tree.xpath(f'//td[contains(@class,"data-col{i} Ta(end) Pstart")]')] for i in [1, 3, 4, 5, 8, 9, 10])


In [3]:
def parseContract(name):
  strike=float(name[-8:])/1000
  tp = 'call' if name[-9]=='C' else 'put'
  expiryDate = datetime.strptime(name[-15:-9], '%y%m%d')
  symbol = name[:-15]
  
parseContract(contractName[0])              

22.5 call 2020-02-21 00:00:00 TDC


In [84]:
def parseTicker(ticker):
    "calls parse Contract for every duration option, \
    returns list of tuples with (expiry date, [options])"
#     url = f'https://finance.yahoo.com/quote/{ticker.lower()}/options'
#     response = requests.get(url)
#     tree = html.fromstring(response.content)
#     expiryDates = tree.xpath("//div[contains(@class, 'option-contract-control drop-down-selector')]")
    
    #Need to automate getting all expiry dates
    urls = ['https://finance.yahoo.com/quote/TDC/options?date=1582243200']
#     ,
#            'https://finance.yahoo.com/quote/TDC/options?date=1584662400',
#            'https://finance.yahoo.com/quote/TDC/options?date=1587081600',
#            'https://finance.yahoo.com/quote/TDC/options?date=1594944000',
#            'https://finance.yahoo.com/quote/TDC/options?date=1608249600']
    for url in urls:
        options = parseContract(url)
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (10,10))
        plt.subplots_adjust(wspace=0.3, hspace=0.25)
        timestamp = int(url.split("=")[1])
        dt_object = datetime.fromtimestamp(timestamp)
        fig.suptitle(f"Implied probabilities for {ticker.upper()} on {datetime.strftime(dt_object, '%B %d, %Y')}")
        for opt, ax in zip(options, (ax1, ax2, ax3, ax4)):
            o = options[opt]
            x_axis = np.array([0] + [i.strike for i in o]+[np.inf])
            step_size = min(x_axis[1:] - x_axis[:-1])
            ax.set_title(f"Implied probabilities of {opt}")
            ax.bar(x_axis[:-1], Euro_Option.make_prob_dist(o), 
                color='tab:orange', label = "Call Vol", alpha = 0.5, width = step_size)
        fig.show()
                     
def parseContract(url):
    'Returns dict of options with bid/ask as premium for each type'
    response = requests.get(url)
    tree = html.fromstring(response.content)
    contractName = [i.text for i in tree.xpath("//td[contains(@class,'data-col0 Ta(start) Pstart')]/a")]
    strike, changePrice, changePercent= \
      ([i.text for i in tree.xpath(f'//td[contains(@class,"data-col{i} Ta(end) Pstart")]/a')] for i in [ 2, 6, 7])
    lastTradeDate, lastPrice, bid, ask, volume, openInterest, impliedVol = \
      ([i.text for i in tree.xpath(f'//td[contains(@class,"data-col{i} Ta(end) Pstart")]')] for i in [1, 3, 4, 5, 8, 9, 10])
    volume = [int(i.replace(",","")) if i!='-' else 0 for i in volume ]
                     
    strike=[float(c[-8:])/1000 for c in contractName]
    tp = ['call' if c[-9]=='C' else 'put' for c in contractName]
    expiryDate = [datetime.strptime(c[-15:-9], '%y%m%d').date() for c in contractName]
    symbol = [c[:-15] for c in contractName]
    tradingLifetimes = [np.busday_count(date.today(), exp) for exp in expiryDate]
                     
    bid_calls = [Euro_Option(tp=t, lifetime = d, strike = s, premium = float(p)) 
               for t,d,s,p,v in zip(tp, tradingLifetimes, strike,bid, volume) 
                 if t == 'call' and v > 0 and float(p) > 0]
    ask_calls = [Euro_Option(tp=t, lifetime = d, strike = s, premium = float(p)) 
               for t,d,s,p,v in zip(tp, tradingLifetimes, strike,ask, volume) 
                 if t == 'put' and v > 0 and float(p) > 0]
    bid_puts = [Euro_Option(tp=t, lifetime = d, strike = s, premium = float(p)) 
               for t,d,s,p,v in zip(tp, tradingLifetimes, strike,bid, volume) 
                 if t == 'call' and v > 0 and float(p) > 0]
    ask_puts = [Euro_Option(tp=t, lifetime = d, strike = s, premium = float(p)) 
               for t,d,s,p,v in zip(tp, tradingLifetimes, strike,ask, volume) 
                 if t == 'put' and v > 0 and float(p) > 0]
    return {'bid_calls':bid_calls, 'ask_calls':ask_calls, 'bid_puts':bid_puts, 'ask_puts':ask_puts}
                     
parseTicker('tdc')

NameError: name 'Euro_Option' is not defined

In [66]:
expiryDates = tree.xpath("//div[contains(@class, 'option-contract-control')]")
expiryDates = tree.xpath('//*[contains(@id,"Col1-1-OptionContracts-Proxy")]')
expiryDates
from datetime import timedelta
timedelta(seconds=2419200).days

optionSeries = namedtuple('optionSeries', ['bid_calls', 'ask_calls', 'bid_puts', 'ask_puts'])
a = optionSeries._make([1,2,3,4])
[i for i in zip(a, a._asdict())]
# [a[i] for i in a._fields]
for i in ('bid_calls', 'ask_calls', 'bid_puts', 'ask_puts'):
    print(f"'{i}':{i}", end=', ')

'bid_calls':bid_calls, 'ask_calls':ask_calls, 'bid_puts':bid_puts, 'ask_puts':ask_puts, 

In [86]:
# strike, changePrice, changePercent= \
#       ([i.text for i in tree.xpath(f'//td[contains(@class,"data-col{i} Ta(end) Pstart")]/a')] for i in [ 2, 6, 7])
#     lastTradeDate, lastPrice, bid, ask, volume, openInterest, impliedVol = \
#       ([i.text for i in tree.xpath(f'//td[contains(@class,"data-col{i} Ta(end) Pstart")]')] for i in [1, 3, 4, 5, 8, 9, 10])
#     volume = [int(i.replace(",","")) if i!='-' else 0 for i in volume ]
'asdf'.upper()

'ASDF'